<a href="https://colab.research.google.com/github/Dhanasree-Rajamani/Data-Mining/blob/main/Timeseries_1_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret 
!pip install pycaret-ts-alpha

In [ ]:
from pycaret.utils import enable_colab
enable_colab()

import time
import numpy as np
import pandas as pd

from pycaret.datasets import get_data
from pycaret.time_series import *
from pycaret.time_series import TSForecastingExperiment

In [ ]:
!pip install Jinja2
!pip install markupsafe==2.0.1

In [ ]:
import jinja2

In [ ]:
fig_kwargs={'renderer': 'notebook'}

In [ ]:
data_set = get_data('pycaret_downloads')

In [ ]:
fh = 12 
fold = 3
data_set['Date'] = pd.to_datetime(data_set['Date'])
data_set.set_index('Date', drop = True, inplace = True)

In [ ]:
ts_obj = TSForecastingExperiment()
ts_obj.setup(data=data_set, fh=fh, fold=fold, fig_kwargs=fig_kwargs)

In [ ]:
best_baseline_models = ts_obj.compare_models(n_select=3)
best_baseline_models

In [ ]:
compare_metrics = ts_obj.pull()

In [ ]:
tm_model = ts_obj.create_model('huber_cds_dt')

In [ ]:
best_tuned_models = [ts_obj.tune_model(tm_model) for model in best_baseline_models]
best_tuned_models

In [ ]:
top_model_metrics = compare_metrics.iloc[0:3]['MAE']
display(top_model_metrics)

top_model_weights = 1 - top_model_metrics/top_model_metrics.sum()
display(top_model_weights)

In [ ]:
blender = ts_obj.blend_models(best_tuned_models, method='voting', weights=top_model_weights.values.tolist())

In [ ]:
y_predict = ts_obj.predict_model(blender)
print(y_predict)
ts_obj.plot_model(estimator=blender)

In [ ]:
final_model_1 = ts_obj.finalize_model(blender)
print(ts_obj.predict_model(final_model_1))
ts_obj.plot_model(final_model_1, fig_kwargs={"height": 600, "width": 1000})

In [ ]:
blender_model = ts_obj.save_model(final_model_1, "my_blender")

In [ ]:
loaded_exp = TSForecastingExperiment()
m = loaded_exp.load_model("my_blender")
# Predictions should be same as before the model was saved and loaded
loaded_exp.predict_model(m)

In [ ]:
ts_obj.predict_model(tm_model)

In [ ]:
# Increased forecast horizon to 2 years instead of the original 1 year
ts_obj.predict_model(tm_model, fh=24)

In [ ]:
ts_obj.plot_model(tm_model, fig_kwargs={"height": 600, "width": 1000})

In [ ]:
ts_obj.plot_model(tm_model, data_kwargs={"alpha": 0.7, "coverage": 0.8}, fig_kwargs={"height": 600, "width": 1000})

In [ ]:
exp = TSForecastingExperiment()
exp.setup(data=data_set, fh=fh, fold=fold, fig_kwargs=fig_kwargs, fold_strategy='sliding', verbose=False)
exp.plot_model(plot="cv", fig_kwargs={"height": 600, "width": 1000})

In [ ]:
exp = TSForecastingExperiment()
exp.setup(data=data_set, fh=fh, fold=fold, fig_kwargs=fig_kwargs, fold_strategy='expanding', verbose=False)
exp.plot_model(plot="cv", fig_kwargs={"height": 600, "width": 1000})

In [ ]:
exp = TSForecastingExperiment()
exp.setup(data=data_set, fh=fh, fold=fold, fig_kwargs=fig_kwargs, fold_strategy='rolling', verbose=False)
exp.plot_model(plot="cv", fig_kwargs={"height": 600, "width": 1000})

In [ ]:
try:
    exp = TSForecastingExperiment()
    exp.setup(data=data_set[:30], fh=12, fold=3, fig_kwargs=fig_kwargs)
except ValueError as error:
    print(error)

In [ ]:
try:
    exp = TSForecastingExperiment()
    exp.setup(data=data_set[:30], fh=6, fold=3, fig_kwargs=fig_kwargs)
except ValueError as error:
    print(error)

In [ ]:
exp = TSForecastingExperiment()
exp.setup(data=data_set, fh=fh, fold=fold, fig_kwargs=fig_kwargs, session_id=42)

In [ ]:
model = exp.create_model("lr_cds_dt")

In [ ]:
# Random Grid Search (default)
tuned_model = exp.tune_model(model)
print(model)
print(tuned_model)

In [ ]:
exp.plot_model([model, tuned_model], data_kwargs={"labels": ["Original", "Tuned"]}, fig_kwargs={"height": 600, "width": 1000})

In [ ]:
tuned_model = exp.tune_model(model, search_algorithm="grid")
print(model)
print(tuned_model)

In [ ]:
ts_obj.plot_model(tm_model, plot='forecast', fig_kwargs={"height": 600, "width": 1000})

In [ ]:
best = ts_obj.compare_models()

In [ ]:
ts_obj.plot_model(best, plot='forecast', fig_kwargs={"height": 600, "width": 1000})

In [ ]:
ts_obj.plot_model(plot="diagnostics", fig_kwargs={"height": 600, "width": 1000})